<a href="https://colab.research.google.com/github/firdarinoa/bangkit-assignment5/blob/develop/Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import math
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.python.data import Dataset
from sklearn import metrics
from matplotlib import cm

In [3]:
cases = pd.read_csv('/content/cases.csv')
cases.head()

,date,acc_tested,new_confirmed,acc_confirmed,acc_negative,being_checked,isolated,new_released,acc_released,new_deceased,acc_deceased,positive_rate,negative_rate,decease_rate,release_rate
0,2-Mar-20,339.0,2,2,335.0,2.0,2,0,0,0,0,0.59%,98.82%,0.00%,0.00%
1,3-Mar-20,341.0,0,2,337.0,2.0,2,0,0,0,0,0.59%,98.83%,0.00%,0.00%
2,4-Mar-20,372.0,0,2,356.0,14.0,2,0,0,0,0,0.54%,95.70%,0.00%,0.00%
3,5-Mar-20,388.0,0,2,371.0,15.0,2,0,0,0,0,0.52%,95.62%,0.00%,0.00%
4,6-Mar-20,450.0,2,4,422.0,24.0,4,0,0,0,0,0.89%,93.78%,0.00%,0.00%


In [4]:
confirmed_acc = pd.read_csv('/content/confirmed_acc.csv')
confirmed_acc.head()

,date,cases
0,1/22/2020,0
1,1/23/2020,0
2,1/24/2020,0
3,1/25/2020,0
4,1/26/2020,0


In [5]:
jakarta = pd.read_csv('/content/jakarta.csv')
jakarta.head()

,date,odp_process,odp_end,odp_total,pdp_process,pdp_end,pdp_total,jakbar,jakpus,jaksel,jaktim,jakut,outsidejakarta,unknown,confirmed,released,deceased,hospitalized,self_isolated
0,3/1/2020,14.0,115.0,129.0,9.0,30.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3/2/2020,14.0,115.0,129.0,9.0,30.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/3/2020,94.0,121.0,215.0,24.0,30.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/4/2020,120.0,121.0,241.0,26.0,30.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3/5/2020,127.0,140.0,267.0,49.0,34.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
keywordtrend = pd.read_csv('/content/keywordtrend.csv')
keywordtrend.head()

,Day,Coronavirus,corona,virus corona,covid,Pneumonia
0,2/1/2020,15,20,8,0,0
1,2/2/2020,22,18,16,0,2
2,2/3/2020,26,23,13,0,2
3,2/4/2020,14,17,7,0,0
4,2/5/2020,15,15,11,0,4


In [7]:
patient = pd.read_csv('/content/patient.csv')
patient.head()

,patient_id,gender,age,nationality,province,current_state,contacted_with,confirmed_date,released_date,deceased_date,hospital
0,1,female,31.0,indonesia,DKI Jakarta,released,NaN,2-Mar-20,13-Mar-20,NaN,RSPI Sulianti Saroso
1,2,female,64.0,indonesia,DKI Jakarta,released,1.0,2-Mar-20,16-Mar-20,NaN,RSPI Sulianti Saroso
2,3,female,33.0,indonesia,DKI Jakarta,released,1.0,6-Mar-20,13-Mar-20,NaN,RSPI Sulianti Saroso
3,4,female,34.0,indonesia,DKI Jakarta,isolated,1.0,6-Mar-20,NaN,NaN,RSPI Sulianti Saroso
4,5,male,55.0,indonesia,DKI Jakarta,isolated,1.0,8-Mar-20,NaN,NaN,RS Persahabatan


In [9]:
province_timeline = pd.read_csv('/content/province_timeline.csv')
province_timeline.head()

,date,aceh,bali,banten,bengkulu,gorontalo,di_yogyakarta,dki_jakarta,jambi,jabar,jateng,jatim,kalbar,kalsel,kalteng,kaltim,kaltara,kep_babel,kep_riau,lampung,maluku,malut,ntb,ntt,papua,papbar,riau,sulbar,sulsel,sulteng,sultra,sulut,sumbar,sumsel,sumut,unknown
0,19-Mar-20,0.0,1.0,39.0,0.0,0.0,4.0,186.0,0.0,40.0,12.0,9.0,2.0,0.0,2.0,3.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,2.0,0.0
1,20-Mar-20,0.0,2.0,4.0,0.0,0.0,0.0,40.0,0.0,1.0,0.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,21-Mar-20,0.0,0.0,4.0,0.0,0.0,1.0,44.0,0.0,14.0,2.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,22-Mar-20,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,4.0,1.0,15.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,23-Mar-20,0.0,3.0,9.0,0.0,0.0,0.0,44.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
